In [51]:
import pandas as pd
from sklearn import linear_model
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [52]:
data = pd.read_pickle('train_data.pkl')

In [53]:
data

,DriverNumber,Time_1,TyreLife_1,LapPercent_1,Compound_1,LapTime_1,Team,Driver,Time_1_1,TyreLife_1_1,...,Compound_2_SOFT,Compound_2_TEST_UNKNOWN,Compound_2_WET,Compound_1_2_,Compound_1_2_HARD,Compound_1_2_INTERMEDIATE,Compound_1_2_MEDIUM,Compound_1_2_SOFT,Compound_1_2_TEST_UNKNOWN,Compound_1_2_WET
0,10,0 days 01:37:09.350000,7.0,0.024500,MEDIUM,0 days 00:01:06.404000,AlphaTauri,GAS,0 days 01:34:35.161000,5.0,...,0,0,0,0,0,0,0,1,0,0
1,11,0 days 01:08:04.652000,14.0,0.010738,SOFT,0 days 00:01:05.512000,Racing Point,PER,0 days 01:05:34.812000,12.0,...,1,0,0,0,0,0,0,1,0,0
2,16,0 days 01:17:25.633000,7.0,0.017095,MEDIUM,0 days 00:01:05.924000,Ferrari,LEC,0 days 01:11:10.147000,2.0,...,1,0,0,0,1,0,0,0,0,0
3,18,0 days 01:02:01.036000,13.0,0.019409,SOFT,0 days 00:01:06.074000,Racing Point,STR,0 days 00:58:17.264000,10.0,...,1,0,0,0,0,0,0,1,0,0
4,20,0 days 01:23:37.572000,7.0,0.016832,SOFT,0 days 00:01:05.907000,Haas F1 Team,MAG,0 days 01:19:25.446000,4.0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,81,0 days 00:18:42.463000,2.0,0.035159,MEDIUM,0 days 00:01:21.496000,McLaren,PIA,0 days 00:17:20.967000,1.0,...,1,0,0,0,0,0,0,1,0,0
1233,10,NaT,NaN,NaN,NaN,NaT,Alpine,GAS,NaT,NaN,...,1,0,0,0,0,0,1,0,0,0
1234,31,NaT,NaN,NaN,NaN,NaT,Alpine,OCO,NaT,NaN,...,1,0,0,0,0,0,1,0,0,0
1235,44,NaT,NaN,NaN,NaN,NaT,Mercedes,HAM,NaT,NaN,...,1,0,0,0,0,0,0,1,0,0


In [54]:
data = data.dropna()

In [55]:
data.columns

Index(['DriverNumber', 'Time_1', 'TyreLife_1', 'LapPercent_1', 'Compound_1',
       'LapTime_1', 'Team', 'Driver', 'Time_1_1', 'TyreLife_1_1',
       'LapPercent_1_1', 'Compound_1_1', 'LapTime_1_1', 'Time_2', 'TyreLife_2',
       'LapPercent_2', 'Compound_2', 'LapTime_2', 'Time_1_2', 'TyreLife_1_2',
       'LapPercent_1_2', 'Compound_1_2', 'LapTime_1_2', 'Position',
       'LapPercent', 'RoundNumber', 'Country', 'Location', 'OfficialEventName',
       'EventDate', 'EventName', 'EventFormat', 'Compound_1_',
       'Compound_1_HARD', 'Compound_1_INTERMEDIATE', 'Compound_1_MEDIUM',
       'Compound_1_SOFT', 'Compound_1_1_', 'Compound_1_1_HARD',
       'Compound_1_1_INTERMEDIATE', 'Compound_1_1_MEDIUM', 'Compound_1_1_SOFT',
       'Compound_1_1_TEST_UNKNOWN', 'Compound_1_1_WET', 'Compound_2_',
       'Compound_2_HARD', 'Compound_2_INTERMEDIATE', 'Compound_2_MEDIUM',
       'Compound_2_SOFT', 'Compound_2_TEST_UNKNOWN', 'Compound_2_WET',
       'Compound_1_2_', 'Compound_1_2_HARD', 'Compound

In [56]:
train_data = data[data['EventDate'] <= np.datetime64('2022-12-31')]
test_data = data[data['EventDate'] > np.datetime64('2022-12-31')]

In [131]:
X_cols =['TyreLife_1', 'LapPercent_1', 'TyreLife_1_1',
       'LapPercent_1_1', 'TyreLife_2',
       'LapPercent_2', 'TyreLife_1_2',
       'LapPercent_1_2', 
       'Compound_1_HARD', 'Compound_1_INTERMEDIATE', 'Compound_1_MEDIUM',
       'Compound_1_SOFT', 'Compound_1_1_HARD',
       'Compound_1_1_INTERMEDIATE', 'Compound_1_1_MEDIUM', 'Compound_1_1_SOFT', 'Compound_1_1_WET',
       'Compound_2_HARD', 'Compound_2_INTERMEDIATE', 'Compound_2_MEDIUM',
       'Compound_2_SOFT', 'Compound_2_WET',
       'Compound_1_2_HARD', 'Compound_1_2_INTERMEDIATE',
       'Compound_1_2_MEDIUM', 'Compound_1_2_SOFT',
       'Compound_1_2_WET']

In [132]:
train_X = train_data[X_cols]
train_y = train_data['LapPercent']

In [133]:
train_X.columns.size

27

In [134]:
train_X

,TyreLife_1,LapPercent_1,TyreLife_1_1,LapPercent_1_1,TyreLife_2,LapPercent_2,TyreLife_1_2,LapPercent_1_2,Compound_1_HARD,Compound_1_INTERMEDIATE,...,Compound_2_HARD,Compound_2_INTERMEDIATE,Compound_2_MEDIUM,Compound_2_SOFT,Compound_2_WET,Compound_1_2_HARD,Compound_1_2_INTERMEDIATE,Compound_1_2_MEDIUM,Compound_1_2_SOFT,Compound_1_2_WET
0,7.0,0.024500,5.0,0.028018,10.0,0.026624,6.0,0.033357,0,0,...,1,0,0,0,0,0,0,0,1,0
1,14.0,0.010738,12.0,0.014765,2.0,0.009968,7.0,0.013809,0,0,...,0,0,0,1,0,0,0,0,1,0
2,7.0,0.017095,2.0,0.020118,3.0,0.015458,6.0,0.027541,0,0,...,0,0,0,1,0,1,0,0,0,0
3,13.0,0.019409,10.0,0.021198,4.0,0.012923,2.0,0.015613,0,0,...,0,0,0,1,0,0,0,0,1,0
4,7.0,0.016832,4.0,0.018869,5.0,0.021367,7.0,0.022300,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,2.0,0.014521,5.0,0.014660,2.0,0.010476,5.0,0.011380,0,0,...,0,0,0,1,0,0,0,0,1,0
1065,2.0,0.000000,5.0,0.006268,5.0,0.007223,2.0,0.008022,0,0,...,0,0,0,1,0,0,0,0,1,0
1066,2.0,0.007330,5.0,0.015964,2.0,0.014669,8.0,0.027435,0,0,...,0,0,0,1,0,0,0,1,0,0
1067,2.0,0.002539,5.0,0.014036,3.0,0.004005,8.0,0.013800,0,0,...,0,0,0,1,0,0,0,1,0,0


In [156]:
model = tree.DecisionTreeRegressor()

In [157]:
params = {'max_depth':[2,3,4,5,6]}
best_model = GridSearchCV(model, param_grid = params).fit(train_X, train_y)

In [158]:
best_model.best_params_

{'max_depth': 2}

In [159]:
test_X = test_data[X_cols]
test_y = test_data['LapPercent']

In [160]:
pred = best_model.predict(test_X)

In [161]:
results = pd.DataFrame({'pred': pred, 'actual': test_y})

In [162]:
results

,pred,actual
1077,0.017535,0.000000
1078,0.017535,0.027567
1079,0.005814,0.001538
1080,0.005814,0.007000
1081,0.017535,0.003255
...,...,...
1227,0.017535,0.058647
1228,0.027731,0.020070
1230,0.017535,0.014366
1231,0.005814,0.039327


In [163]:
results.describe()

,pred,actual
count,145.000000,145.000000
mean,0.019228,0.017357
std,0.007972,0.037934
min,0.005814,0.000000
25%,0.017535,0.007536
50%,0.017535,0.013437
75%,0.027731,0.019589
max,0.074820,0.455956


In [164]:
joblib.dump(best_model, 'model.pkl')

['model.pkl']